In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pandas
!pip install tqdm
!pip install value-nlp

In [ ]:
import pandas as pd
import json
from multivalue import Dialects
import spacy
import stanza

inde = Dialects.IndianDialect()

def translate_text(text):
    try:
        return inde.transform(text)
    except Exception as e:
        print(f"Error in translation: {e}")
        return text  

def process_csv(file_path, column_to_translate):
    df = pd.read_csv(file_path)
    translated_column = []
    total_rows = len(df)

    for idx, text in enumerate(df[column_to_translate], start=1):
        translated_text = translate_text(text)
        translated_column.append(translated_text)
        print(f"Processed row {idx}/{total_rows}")

    df[f"IndE ({column_to_translate})"] = translated_column
    output_path = f"/content/drive/MyDrive/!!Multi-AAVENUE/Translated Datasets/Multi-VALUE/IndE/{file_path.split('/')[-1].replace('.csv', '_MVIndE.csv')}"
    df.to_csv(output_path, index=False)
    print(f"Translated file saved to: {output_path}")

def process_json(file_path, key_to_translate):
    with open(file_path, "r") as file:
        data = json.load(file)
    translated_data = []
    total_items = len(data["propositional_logic"]["samples"])

    for idx, item in enumerate(data["propositional_logic"]["samples"], start=1):
        item_copy = item.copy()
        item_copy[f"IndE ({key_to_translate})"] = translate_text(item[key_to_translate])
        translated_data.append(item_copy)
        print(f"Processed row {idx}/{total_items}")

    data["propositional_logic"]["samples"] = translated_data
    output_path = f"/content/drive/MyDrive/!!Multi-AAVENUE/Translated Datasets/Multi-VALUE/IndE/{file_path.split('/')[-1].replace('.json', '_MVIndE.json')}"
    with open(output_path, "w") as file:
        json.dump(data, file, indent=4)
    print(f"Translated file saved to: {output_path}")

file_translation_map = {
    "/content/drive/MyDrive/!!Multi-AAVENUE/Original Datasets/FOLIO(1000).csv": "Premises",
    "/content/drive/MyDrive/!!Multi-AAVENUE/Original Datasets/GSM8K(1000).csv": "Original",
    "/content/drive/MyDrive/!!Multi-AAVENUE/Original Datasets/HumanEVAL(164).csv": "Prompt",
    "/content/drive/MyDrive/!!Multi-AAVENUE/Original Datasets/Logic Bench MCQ(480).json": "context",
    "/content/drive/MyDrive/!!Multi-AAVENUE/Original Datasets/Logic Bench YN(500).json": "context",
    "/content/drive/MyDrive/!!Multi-AAVENUE/Original Datasets/MBPP(374).csv": "Original",
    "/content/drive/MyDrive/!!Multi-AAVENUE/Original Datasets/SVAMP(700).csv": "Original"
}

for file_path, column_or_key in file_translation_map.items():
    if file_path.endswith(".csv"):
        process_csv(file_path, column_or_key)
    elif file_path.endswith(".json"):
        process_json(file_path, column_or_key)

print("All translations completed and saved.")